In [1]:
import random, os
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from glob import glob
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
path = '../dataset/'

In [3]:
def load_data():
    dataset = []
    filenames = os.listdir(path)
    for file in filenames:
        pic = Image.open(path + file)
        pic = np.array(pic)[:, :, 1]
        dataset.append(pic)
    print(np.array(dataset).shape)
    return dataset

In [4]:
dataset = load_data()

(16, 28, 28)


In [5]:
input_img = Input(shape=(1, 28, 28))

In [6]:
x = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(input_img) #nb_filter, nb_row, nb_col
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
encoded = MaxPooling2D((2, 2), border_mode='same')(x)

x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = UpSampling2D((2, 2))(x)

x = Convolution2D(16, 3, 3, activation='relu', border_mode='valid')(x) 

x = UpSampling2D((2, 2))(x)
decoded = Convolution2D(1, 5, 5, activation='sigmoid', border_mode='same')(x)

/home/minji/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
  """Entry point for launching an IPython kernel.
/home/minji/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
  
/home/minji/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/minji/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
  after removing the cwd from sys.path.
/home/minji/.conda/envs/tensor/lib/python3.6/site-pa

In [7]:
autoencoder = Model(input_img, decoded)

In [8]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [9]:
x_train = x_test = dataset

In [10]:
x_train = np.array(x_train).astype('float32')/255.
x_test = np.array(x_test).astype('float32')/255.

In [11]:
x_train = np.reshape(x_train, (len(x_train), 1, 28, 28))
x_test = np.reshape(x_test, (len(x_test), 1, 28, 28))

In [12]:
autoencoder.fit(x_train, x_train, nb_epoch=50, batch_size=128, shuffle=True, validation_data=(x_test, x_test), verbose=1)

/home/minji/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


ValueError: Error when checking target: expected conv2d_7 to have shape (4, 28, 1) but got array with shape (1, 28, 28)